# Importações

In [ ]:
#!pip install scikit-learn shap pandas numpy matplotlib xgboost seaborn 

In [ ]:
import pickle
import joblib
import shap
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np  
import os
import sys

# Pré-processamento e pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

# Modelos de ML
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Validação e busca de hiperparâmetros
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from scipy.stats import randint, uniform, loguniform

# Métricas
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score

# Transformadores personalizados
src_path = os.path.abspath(os.path.join("..", "Streamlit", "src"))
if src_path not in sys.path:
    sys.path.append(src_path)

from custom_transformers import (
    DateFeatureExtractor,
    CapTransformer,
    RareCategoryGrouper,
    TopNMultiLabelTransformer,
    BudgetRuntimeRatioTransformer
)

# Carregando dataset

In [ ]:
df = pd.read_csv('../data/filmes_filtrados.csv')
df = df.drop(columns=['popularity'])

# Dividindo os dados
- X_train, X_test, y_train, y_test

In [ ]:
mean_impute_cols = ['budget', 'runtime', 'budget_runtime_ratio']
mode_impute_cols = ['year', 'month']
categorical_col = ['original_language']
date_column = 'release_date'

X = df.drop(columns=['vote_average'])
y = df['vote_average']

# Dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)

# Transformadores
- MultiLabelBinarize -> Generos Cinematográficos, Produtora e Atores
- Data -> Extrai o ano e o mês das datas
- Outliers -> Cap, Log + Cap, Winsorizer
- Agrupamento de categorias raras -> original language
- Criação da feature budget_runtime_ratio

In [ ]:
# Transformers personalizados por coluna
credits_transformer = TopNMultiLabelTransformer(top_n=60, prefix='credits')
genres_transformer = TopNMultiLabelTransformer(top_n=12, prefix='genre')
prod_companies_transformer = TopNMultiLabelTransformer(top_n=50, prefix='prod_company')

# Treinamento
- Modelo XGBRegressor
- Modelo SVR
- Modelo RandomForest

### Definindo Kf

In [ ]:
# Definindo o KFold para o cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=87)

### XGBRegressor

In [ ]:
# Sub-pipelines de pré-processamento
mean_numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('cap', CapTransformer(columns=mean_impute_cols)),
    ('scale', StandardScaler())
])

mode_numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cap', CapTransformer(columns=mode_impute_cols)),
    ('scale', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('rare', RareCategoryGrouper(column='original_language', top_n=10, other_label='Other')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Pré-processador principal
preprocessor = ColumnTransformer([
    ('mean_numeric', mean_numeric_pipeline, mean_impute_cols),
    ('mode_numeric', mode_numeric_pipeline, mode_impute_cols),
    ('categorical', cat_pipeline, categorical_col),
    ('credits', credits_transformer, 'credits'),
    ('genres', genres_transformer, 'genres'),
    ('prod_companies', prod_companies_transformer, 'production_companies')
], remainder='drop')

# Pipeline final e completo
pipeline = Pipeline([
    ('budget_ratio', BudgetRuntimeRatioTransformer()),
    ('date', DateFeatureExtractor(date_column=date_column)),
    ('preprocess', preprocessor),
    ('regressor', XGBRegressor(objective='reg:squarederror', random_state=87, n_jobs=-1))
])

pipeline.set_output(transform="pandas")

# Definição dos parâmetros para a busca
param_dist = {
    'regressor__n_estimators': randint(100, 300),
    'regressor__max_depth': randint(3, 10),
    'regressor__learning_rate': uniform(0.01, 0.3),
    'regressor__subsample': uniform(0.7, 0.3),
    'regressor__colsample_bytree': uniform(0.7, 0.3),
}

# Treinamento com RandomizedSearchCV
XGBRegressor_rand = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=30,
    cv=kf,
    scoring='r2',
    random_state=87,
    verbose=2,
    n_jobs=-1
)

XGBRegressor_rand.fit(X_train, y_train)

# Exibição e salvamento dos resultados
print("Melhores parâmetros:", XGBRegressor_rand.best_params_)
print("Melhor R² (validação cruzada):", XGBRegressor_rand.best_score_)

### SVR

In [ ]:
# --- 2. Definição dos Sub-pipelines de Pré-processamento ---
mean_numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('cap', CapTransformer(columns=mean_impute_cols)),
    ('scale', StandardScaler())
])

mode_numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cap', CapTransformer(columns=mode_impute_cols)),
    ('scale', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('rare', RareCategoryGrouper(column='original_language', top_n=10, other_label='Other')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# --- 3. Construção do Pré-processador Principal ---
# Este preprocessor é reutilizado do setup do XGBoost
preprocessor = ColumnTransformer([
    ('mean_numeric', mean_numeric_pipeline, mean_impute_cols),
    ('mode_numeric', mode_numeric_pipeline, mode_impute_cols),
    ('categorical', cat_pipeline, categorical_col),
    ('credits', credits_transformer, 'credits'),
    ('genres', genres_transformer, 'genres'),
    ('prod_companies', prod_companies_transformer, 'production_companies')
], remainder='drop')


# --- 4. Construção do Pipeline Final para o SVR ---
pipeline_svr = Pipeline([
    # As etapas iniciais são idênticas ao pipeline do XGBoost
    ('budget_ratio', BudgetRuntimeRatioTransformer()),
    ('date', DateFeatureExtractor(date_column=date_column)),
    
    # A etapa de pré-processamento reutiliza o mesmo preprocessor robusto
    ('preprocess', preprocessor),
    
    # A única diferença é o estimador final
    ('regressor', SVR())
])

pipeline_svr.set_output(transform="pandas")


# --- 5. Otimização e Treinamento do SVR ---
# A distribuição de parâmetros e a busca continuam as mesmas
param_dist_svr = {
    'regressor__kernel': ['rbf', 'linear'],
    'regressor__C': loguniform(1e-2, 1e2),
    'regressor__epsilon': uniform(0.01, 0.3),
    'regressor__gamma': ['scale', 'auto']
}

svm_rand = RandomizedSearchCV(
    estimator=pipeline_svr,
    param_distributions=param_dist_svr,
    n_iter=30,
    cv=kf,
    scoring='r2',
    random_state=87,
    verbose=2,
    n_jobs=-1
)

svm_rand.fit(X_train, y_train)

# --- 6. Exibição e Salvamento dos Resultados ---
print("Melhores parâmetros SVR:", svm_rand.best_params_)
print("Melhor R² (CV) SVR:", svm_rand.best_score_)

### RandomForest

In [ ]:
# --- 2. Definição dos Sub-pipelines de Pré-processamento ---
mean_numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('cap', CapTransformer(columns=mean_impute_cols)),
    ('scale', StandardScaler())
])

mode_numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cap', CapTransformer(columns=mode_impute_cols)),
    ('scale', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('rare', RareCategoryGrouper(column='original_language', top_n=10, other_label='Other')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# --- 3. Construção do Pré-processador Principal ---
# Este preprocessor é reutilizado dos setups anteriores
preprocessor = ColumnTransformer([
    ('mean_numeric', mean_numeric_pipeline, mean_impute_cols),
    ('mode_numeric', mode_numeric_pipeline, mode_impute_cols),
    ('categorical', cat_pipeline, categorical_col),
    ('credits', credits_transformer, 'credits'),
    ('genres', genres_transformer, 'genres'),
    ('prod_companies', prod_companies_transformer, 'production_companies')
], remainder='drop')


# --- 4. Construção do Pipeline Final para o Random Forest ---
pipeline_rf = Pipeline([
    # Etapas iniciais idênticas aos outros pipelines
    ('budget_ratio', BudgetRuntimeRatioTransformer()),
    ('date', DateFeatureExtractor(date_column=date_column)),
    
    # Reutiliza o mesmo preprocessor robusto
    ('preprocess', preprocessor),
    
    # A única diferença é o estimador final
    ('regressor', RandomForestRegressor(random_state=87, n_jobs=-1))
])

pipeline_rf.set_output(transform="pandas")


# --- 5. Otimização e Treinamento do Random Forest ---
# A distribuição de parâmetros e a busca continuam as mesmas
param_dist_rf = {
    'regressor__n_estimators': randint(100, 500),
    'regressor__max_depth': randint(3, 20),
    'regressor__min_samples_split': randint(2, 10),
    'regressor__min_samples_leaf': randint(1, 10),
    'regressor__max_features': ['sqrt', 'log2', None]
}

rf_rand = RandomizedSearchCV(
    estimator=pipeline_rf,
    param_distributions=param_dist_rf,
    n_iter=30,
    cv=kf,
    scoring='r2',
    error_score='raise',
    random_state=87,
    verbose=2,
    n_jobs=-1
)

rf_rand.fit(X_train, y_train)

# --- 6. Exibição e Salvamento dos Resultados ---
print("Melhores parâmetros RF:", rf_rand.best_params_)
print("Melhor R² (CV) RF:", rf_rand.best_score_)

# Avaliação dos modelos

## Treino

In [ ]:
def evaluate_model_train(model_name, model, X_train, y_train):
    y_pred_train = model.predict(X_train)
    return {
        'Modelo': model_name,
        'R²': r2_score(y_train, y_pred_train),
        'MAE': mean_absolute_error(y_train, y_pred_train),
        'RMSE': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'MedAE': median_absolute_error(y_train, y_pred_train)
    }

In [ ]:
evaluate_model_train("XGBRegressor", XGBRegressor_rand.best_estimator_, X_train, y_train)

In [ ]:
results_train = []
results_train.append(evaluate_model_train("XGBRegressor", XGBRegressor_rand.best_estimator_, X_train, y_train))
results_train.append(evaluate_model_train("RandomForestRegressor", rf_rand.best_estimator_, X_train, y_train))
results_train.append(evaluate_model_train("SVR", svm_rand.best_estimator_, X_train, y_train))
results_df_train = pd.DataFrame(results_train)
print(results_df_train)

## Teste

In [ ]:
def evaluate_model_test(model_name, model, X_test, y_test):
    y_pred_test = model.predict(X_test)
    return {
        'Modelo': model_name,
        'R²': r2_score(y_test, y_pred_test),
        'MAE': mean_absolute_error(y_test, y_pred_test),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'MedAE': median_absolute_error(y_test, y_pred_test)
    }

In [ ]:
evaluate_model_test("XGBRegressor", XGBRegressor_rand.best_estimator_, X_test, y_test)

In [ ]:
results_test = []
results_test.append(evaluate_model_test("XGBRegressor", XGBRegressor_rand.best_estimator_, X_test, y_test))
results_test.append(evaluate_model_test("RandomForestRegressor", rf_rand.best_estimator_, X_test, y_test))
results_test.append(evaluate_model_test("SVR", svm_rand.best_estimator_, X_test, y_test))
results_df_test = pd.DataFrame(results_test)
print(results_df_test)

# Graficos

## Shap

### Ingles

In [ ]:
# 1. Obter o melhor pipeline treinado
best_pipeline = XGBRegressor_rand.best_estimator_

# 2. Criar um pipeline APENAS com os passos de pré-processamento
preprocessing_pipeline = Pipeline(best_pipeline.steps[:-1])
X_train_transformed = preprocessing_pipeline.transform(X_train)


# 3. Obter os nomes das features do ColumnTransformer (que é o passo 'preprocess')
col_transformer = best_pipeline.named_steps['preprocess']
feature_names = col_transformer.get_feature_names_out()

# 4. Criar um DataFrame com os dados transformados para melhor visualização no SHAP
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=feature_names)

# 5. Extrair o modelo final e calcular os valores SHAP
xgb_model = best_pipeline.named_steps['regressor']
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer(X_train_transformed_df) # A API moderna do SHAP usa o explainer como uma função
print("Valores SHAP calculados com sucesso. Gerando gráficos...")

# 6. Gerar os gráficos
plt.figure()
plt.title("SHAP Summary Plot (Beeswarm)")
shap.summary_plot(shap_values, X_train_transformed_df, show=False)
plt.tight_layout() 
plt.show()
plt.figure()
plt.title("Importância Média das Top 40 Features (SHAP Bar Plot)")
shap.summary_plot(shap_values, X_train_transformed_df, plot_type="bar", show=False, max_display=40)
plt.tight_layout()
plt.show()

### Traduzido

In [ ]:
# 1. Obter o melhor pipeline treinado
best_pipeline = XGBRegressor_rand.best_estimator_

# 2. Criar um pipeline APENAS com os passos de pré-processamento
preprocessing_pipeline = Pipeline(best_pipeline.steps[:-1])
X_train_transformed = preprocessing_pipeline.transform(X_train)

# 3. Obter os nomes das features do ColumnTransformer (que é o passo 'preprocess')
col_transformer = best_pipeline.named_steps['preprocess']
feature_names = col_transformer.get_feature_names_out()

# 4. Criar um DataFrame com os dados transformados para melhor visualização no SHAP
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=feature_names)

# 5. Extrair o modelo final e calcular os valores SHAP
xgb_model = best_pipeline.named_steps['regressor']
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer(X_train_transformed_df)
print("Valores SHAP calculados com sucesso.")

# -------------------------------------------------------------------------- #
#                        ✨ INÍCIO DA MODIFICAÇÃO ✨                         #
# -------------------------------------------------------------------------- #

# 6. Criar o dicionário de tradução e preparar os nomes para o gráfico

print("Traduzindo nomes das features para o português...")

# Dicionário para mapear os nomes originais para português (AJUSTADO PARA O NOVO GRÁFICO)
mapa_traducao = {
    'mean_numeric__runtime': 'Duração (Média)',
    'mode_numeric__year': 'Ano de Lançamento (Moda)',
    'genres__genre_Drama': 'Gênero: Drama',
    'genres__genre_Outros': 'Gênero: Outros',
    'genres__genre_Action': 'Gênero: Ação',
    'genres__genre_Horror': 'Gênero: Terror',
    'categorical__original_language_en': 'Idioma: Inglês',
    'mean_numeric__budget_runtime_ratio': 'Orçamento / Duração (Média)',
    'mean_numeric__budget': 'Orçamento (Média)',
    'genres__genre_Science Fiction': 'Gênero: Ficção Científica',
    'mode_numeric__month': 'Mês de Lançamento (Moda)',
    'genres__genre_Adventure': 'Gênero: Aventura',
    'genres__genre_Crime': 'Gênero: Crime',
    'genres__genre_Comedy': 'Gênero: Comédia',
    'genres__genre_Thriller': 'Gênero: Suspense',
    'categorical__original_language_ja': 'Idioma: Japonês',
    'categorical__original_language_Other': 'Idioma: Outro',
    'genres__genre_Romance': 'Gênero: Romance',
    'categorical__original_language_hi': 'Idioma: Hindi',
    'genres__genre_Family': 'Gênero: Família'
}


# Renomeia as colunas do DataFrame para obter uma lista de nomes traduzidos
X_train_transformed_df_traduzido = X_train_transformed_df.rename(columns=mapa_traducao)
nomes_traduzidos = X_train_transformed_df_traduzido.columns.tolist()


# -------------------------------------------------------------------------- #
#                          ✨ FIM DA MODIFICAÇÃO ✨                           #
# -------------------------------------------------------------------------- #

# 7. Gerar os gráficos com os nomes traduzidos (MÉTODO CORRIGIDO)
print("Gerando gráficos com nomes traduzidos...")

# --- Gráfico Beeswarm ---
plt.figure()
plt.title("SHAP Summary Plot (Beeswarm) - Traduzido")
shap.summary_plot(
    shap_values,
    features=X_train_transformed_df,   # Usa o DataFrame ORIGINAL para os valores
    feature_names=nomes_traduzidos,     # Usa a lista de NOMES TRADUZIDOS para as etiquetas
    show=False
)
plt.tight_layout() 
plt.show()

# --- Gráfico de Barras (Bar Plot) ---
plt.figure()
plt.title("Importância Média das Top 40 Features (SHAP Bar Plot) - Traduzido")
shap.summary_plot(
    shap_values,
    features=X_train_transformed_df,   # Usa o DataFrame ORIGINAL para os valores
    feature_names=nomes_traduzidos,     # Usa a lista de NOMES TRADUZIDOS para as etiquetas
    plot_type="bar",
    show=False,
    max_display=40
)
plt.tight_layout()
plt.show()

## Importance

In [ ]:
feature_importance = np.abs(shap_values.values).mean(axis=0)
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'SHAP Importance': feature_importance
})

sorted_importance_df = importance_df.sort_values(
    by='SHAP Importance', 
    ascending=False
).reset_index(drop=True)

print("Ranking de Importância das Features (baseado em SHAP):")
pd.set_option('display.max_rows', None)
print(len(sorted_importance_df))
display(sorted_importance_df) 

# Gerando Pickle

In [ ]:
melhor_pipeline_final = XGBRegressor_rand.best_estimator_
nome_arquivo_modelo = 'best_model_sem_popularidade.pkl'

In [ ]:
#joblib.dump(melhor_pipeline_final, nome_arquivo_modelo)